Imports

In [1]:
import sys
sys.path.append("../src") 

In [2]:
import pandas as pd
from data.wav_data_reader import WavDataReader
from data.signal_dataset import SignalDataset
from features.wav_feature_extractor import WavFeatureExtractor
from features.features_dataset import FeaturesDataset
import data.preparation_eurythmy_data as ped
from models.fully_connected_classifier import FullyConnectedClassifier
from evaluation.hyperparameter_tuner import HyperparameterTuner

Reader

In [3]:
test_folder= r"..\data\interim\testing"

In [4]:
reader= WavDataReader(folder= test_folder)
signals, ids= reader.get_values_and_keys()
meas_df= ped.return_meas_labels_by_keys(ids)

Signal Dataset

In [5]:
#Initialize
signal_dataset= SignalDataset(signals= signals, features= meas_df)

#Process Data
signal_dataset.standardize_signals("zscore")
signal_dataset.segment_signals(segment_duration=1)
signal_dataset.remove_constant_signals()

#Add eurythmy letter data to the features
signal_dataset.features= ped.add_meas_letters(signal_dataset.features)

,id_measurement,id_performance,datetime,plant,generation,num_eurythmy
0,1,1,2023-04-29,salad,1,1
1,15,3,2023-05-09,salad,1,1
2,44,8,2023-05-19,tomato,1,1
3,129,23,2023-06-12,tomato,1,1
4,252,44,2023-07-11,basil,1,0
5,278,48,2023-07-17,basil,1,0


In [8]:
# Function in ped that returns indexes to keep for each question and targets
# Function in signal dataset that implements?

In [14]:
len(signal_dataset.features)

1348

In [9]:
indexes, target_list= ped.get_targets_rq2_what_letter(signal_dataset.features)

In [13]:
print(target_list)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 

In [10]:
klk

NameError: name 'klk' is not defined

In [ ]:
signals, eurythmy_features= signal_dataset.get_data()

Feature Extractor

In [ ]:
%%time
extractor= WavFeatureExtractor(sample_rate= 10000)
all_features, feature_labels= extractor.extract_features_multiple_waveforms(waveforms= signals)

In [ ]:
feature_labels= feature_labels.append(eurythmy_labels)
all_features.append(eurythmy_features)

Features Dataset

In [ ]:
feat_dataset= FeaturesDataset(features= all_features, feature_labels= feature_labels)
feat_dataset.save_to_csv("raw_dataset.csv")
feat_dataset.process_features()
feat_dataset.shape()

In [ ]:
num_feat= len(feat_dataset.get_labels())
feat_dataset.head()
feat_dataset.save_to_csv("processed_dataset.csv")

In [ ]:
train_loader, val_loader, test_loader = feat_dataset.split_dataset_in_loaders(test_size=0.3, val_size=0.5, random_state=42)

## Search

In [ ]:
input_size= num_feat
output_size= 2
num_epochs = 5

param_grid = {
    'learning_rate': [0.1, 0.01],
    'dense_units': [64, 128],
    'dense_layers': [1, 2],
    'dropout_rate': [0.25, 0.5]
}

tuner = HyperparameterTuner(FullyConnectedClassifier, param_grid, train_loader, val_loader, num_epochs, input_size, output_size)
best_params, all_results = tuner.tune()

In [ ]:
print("Best Hyperparameters: \n", best_params)

In [ ]:
all_results.head(15)

In [ ]:
klk

## Model

In [ ]:
num_epochs = 5  # Number of training epochs

In [ ]:
model = FullyConnectedClassifier(
    input_size=5,  # Example input size
    hidden_layers=[128, 64],  # Two hidden layers with 128 and 64 units respectively
    output_size=2,  # Example output size
    dropout_rate=0.5,
    learning_rate=0.001
)

In [ ]:
model.train_model(train_loader, val_loader, num_epochs)

In [ ]:
test_predictions = model.predict(test_loader)

In [ ]:
actual_labels = []
for _, labels in test_loader:
    actual_labels.extend(labels.tolist())

In [ ]:
correct_predictions = sum(p == t for p, t in zip(test_predictions, actual_labels))
accuracy = correct_predictions / len(actual_labels)
print(f"Accuracy: {accuracy:.4f}")


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Assuming your task is a classification task
precision = precision_score(actual_labels, test_predictions, average='macro')
recall = recall_score(actual_labels, test_predictions, average='macro')
f1 = f1_score(actual_labels, test_predictions, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(actual_labels, test_predictions)
print("Confusion Matrix:\n", conf_matrix)
